In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import rasterio
# import matplotlib.pyplot as plt
# from rasterio.plot import show as rio_show
# import matplotlib as mpl
import pandas as pd
import geopandas as gpd
import numpy as np

In [3]:
df = pd.read_csv('../results_2022.csv', dtype=str)
df

,Date,Duration (days),Template size (px),Template size (m),Pixel spacing (px),Pixel spacing (m),Prefilter,Subpixel,Software,Vx,Vy,SAV-uncertainty-x,SAV-uncertainty-y,SAV-peak-x,SAV-peak-y,LSR-uncertainty-nm,LSR-uncertainty-sh
0,Sen2-20180304-20180314,10,48,480,12,120,Gau,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,0.32637951498361006,0.35901746648197114,-0.07337931716612034,0.17662068283387966,0.0017456245187072372,0.0013862312354439828
1,Sen2-20180304-20180314,10,48,480,12,120,NAOF,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,0.2904061391872022,0.299206325223178,-0.05369981396402412,0.16505018603597588,0.009696277915313002,0.009108624708324334
2,Sen2-20180304-20180314,10,48,480,12,120,None,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,0.4480050686250796,0.47890196990956785,-0.031426549357755906,0.1095515493577558,0.012744702296493292,0.012744702296493292
3,Sen2-20180304-20180314,10,48,480,1,10,Gau,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,0.19932657031566106,0.21744716761708482,-0.0685401991004746,0.20562059730142374,0.0075318103275894,0.0075318103275894
4,Sen2-20180304-20180314,10,48,480,1,10,NAOF,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,0.170416834102238,0.18288635854874324,-0.05834349185116494,0.16871952444650518,0.05414784514500587,0.05414784514500587
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,Sen2-20180508-20180627,50,64,640,8,80,Gau,pyrUP,autoRIFT,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,0.17952278944131322,0.2593106958596746,0.04335100854352797,-0.05744697809470645,0.007606484754625452,0.007606484754625452
168,Sen2-20180508-20180627,50,32,320,4,40,NAOF,pyrUP,autoRIFT,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,0.11496105665321626,0.15063862595938682,0.041464175857468366,-0.016464174553616817,0.006968377551435475,0.006546051639227265
169,Sen2-20180508-20180627,50,32,320,8,80,NAOF,pyrUP,autoRIFT,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,0.07502428079962462,0.1110359355834444,0.0405009727221011,-0.021502913882219477,0.0032284029549105872,0.0033360163867409403
170,Sen2-20180508-20180627,50,64,640,4,40,NAOF,pyrUP,autoRIFT,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,0.07284891177652728,0.11073034590032146,0.04041395796117722,-0.015413956657325617,0.004378758498193732,0.004237508224058451


In [4]:
gps_files = ['/home/jovyan/Projects/PX_comparison/GPS/Kaskawulsh_2018-04-05_to_2018-03-04_GPS', 
             '/home/jovyan/Projects/PX_comparison/GPS/Kaskawulsh_2018-08-18_to_2018-08-02_GPS',
             '/home/jovyan/Projects/PX_comparison/GPS/Kaskawulsh_2018-03-14_to_2018-03-04_GPS',
             '/home/jovyan/Projects/PX_comparison/GPS/Kaskawulsh_v2_2018-05-08_to_2018-06-27_GPS.csv']

datestrs = ['LS8-20180304-20180405', 'LS8-20180802-20180818', 'Sen2-20180304-20180314', 'Sen2-20180508-20180627']

In [5]:
for gps_file, datestr in zip(gps_files, datestrs):
    gps = pd.read_csv(gps_file) 
    gps = gpd.GeoDataFrame(gps, geometry=gpd.points_from_xy(gps['end_easting'], gps['end_northing']), crs='EPSG:32607')
    gps_xy = list(gps[['end_easting', 'end_northing']].to_records(index=False))
    if datestr == 'Sen2-20180508-20180627':
        gps_xy[-2] = (601790.43877479, 6733753.77267354)  # it should be (nan, nan), (nan, nan) but nans does not work with rio.sample
        gps_xy = gps_xy[:-1]
    print(gps_xy)
    
    df_s = df.loc[df['Date'] == datestr]
    for idx, row in df_s.iterrows():
        vx_grid = rasterio.open(row.Vx)
        vy_grid = rasterio.open(row.Vy)
        sample_gen_vx = vx_grid.sample(gps_xy)
        vx_sampled = np.array([float(record) for record in sample_gen_vx])
        sample_gen_vy = vy_grid.sample(gps_xy)
        vy_sampled = np.array([float(record) for record in sample_gen_vy])
        vx_sampled[vx_sampled < -9998] = np.nan
        vy_sampled[vy_sampled < -9998] = np.nan
        vx_sampled -= float(df.loc[idx, 'SAV-peak-x'])
        vy_sampled -= float(df.loc[idx, 'SAV-peak-y'])
        v_sampled = np.sqrt(vx_sampled ** 2 + vy_sampled ** 2)
        df.loc[idx, 'pt0_velodiff'] = v_sampled[0] - np.abs(gps.loc[0, 'velocity (m/d)'])
        df.loc[idx, 'pt1_velodiff'] = v_sampled[1] - np.abs(gps.loc[1, 'velocity (m/d)'])
        df.loc[idx, 'pt2_velodiff'] = v_sampled[2] - np.abs(gps.loc[2, 'velocity (m/d)'])

[(621306.41954208, 6738829.50233354), (610435.5249175, 6737129.57698521), (601733.22946583, 6733710.66504834)]
[(621363.01607688, 6738895.12164604), (610506.52739125, 6737089.56006354), (601790.43877479, 6733753.77267354)]
[(621306.41954208, 6738829.50233354), (610435.5249175, 6737129.57698521), (601733.22946583, 6733710.66504834)]
[(621348.11544947, 6738880.22819989), (610481.28682665, 6737102.95371238), (601790.43877479, 6733753.77267354)]


In [6]:
df

,Date,Duration (days),Template size (px),Template size (m),Pixel spacing (px),Pixel spacing (m),Prefilter,Subpixel,Software,Vx,Vy,SAV-uncertainty-x,SAV-uncertainty-y,SAV-peak-x,SAV-peak-y,LSR-uncertainty-nm,LSR-uncertainty-sh,pt0_velodiff,pt1_velodiff,pt2_velodiff
0,Sen2-20180304-20180314,10,48,480,12,120,Gau,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,0.32637951498361006,0.35901746648197114,-0.07337931716612034,0.17662068283387966,0.0017456245187072372,0.0013862312354439828,0.088111,-0.026778,0.950114
1,Sen2-20180304-20180314,10,48,480,12,120,NAOF,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,0.2904061391872022,0.299206325223178,-0.05369981396402412,0.16505018603597588,0.009696277915313002,0.009108624708324334,-0.080811,-0.009888,NaN
2,Sen2-20180304-20180314,10,48,480,12,120,None,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,0.4480050686250796,0.47890196990956785,-0.031426549357755906,0.1095515493577558,0.012744702296493292,0.012744702296493292,-0.193226,-0.196510,4.550476
3,Sen2-20180304-20180314,10,48,480,1,10,Gau,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,0.19932657031566106,0.21744716761708482,-0.0685401991004746,0.20562059730142374,0.0075318103275894,0.0075318103275894,-0.012384,-0.022131,0.912726
4,Sen2-20180304-20180314,10,48,480,1,10,NAOF,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,0.170416834102238,0.18288635854874324,-0.05834349185116494,0.16871952444650518,0.05414784514500587,0.05414784514500587,-0.139254,-0.004308,7.479813
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,Sen2-20180508-20180627,50,64,640,8,80,Gau,pyrUP,autoRIFT,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,0.17952278944131322,0.2593106958596746,0.04335100854352797,-0.05744697809470645,0.007606484754625452,0.007606484754625452,0.004121,NaN,NaN
168,Sen2-20180508-20180627,50,32,320,4,40,NAOF,pyrUP,autoRIFT,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,0.11496105665321626,0.15063862595938682,0.041464175857468366,-0.016464174553616817,0.006968377551435475,0.006546051639227265,-0.048683,NaN,NaN
169,Sen2-20180508-20180627,50,32,320,8,80,NAOF,pyrUP,autoRIFT,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,0.07502428079962462,0.1110359355834444,0.0405009727221011,-0.021502913882219477,0.0032284029549105872,0.0033360163867409403,-0.051604,NaN,NaN
170,Sen2-20180508-20180627,50,64,640,4,40,NAOF,pyrUP,autoRIFT,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,0.07284891177652728,0.11073034590032146,0.04041395796117722,-0.015413956657325617,0.004378758498193732,0.004237508224058451,-0.073796,NaN,NaN


In [7]:
df.to_csv('../results_2022.csv', index=False)

In [8]:
df.loc[idx, 'OI-peak-x']

'0.03750000149011612'